In [83]:
import pandas as pd
import altair as alt
import numpy as np

from scipy.signal import savgol_filter
import loess.loess_1d
import scipy.optimize
import scipy.interpolate

In [4]:
data1 = pd.read_json('measurements_2.js')

In [8]:
data2 = pd.read_json('/home/mu/.local/share/autobright/measurements.js')

In [10]:
data = pd.concat([data1, data2])

In [11]:
data

,datetime,reading,brightness
0,2021-11-17 09:16:27.733,359,11
1,2021-11-17 12:43:34.860,279,9
2,2021-11-17 12:50:02.146,273,8
3,2021-11-17 14:17:43.697,291,7
4,2021-11-17 14:42:29.204,197,6
...,...,...,...
19,2021-11-21 15:12:35.363,224,5
20,2021-11-22 18:05:38.775,741,5
21,2022-05-15 12:45:56.401,525,40
22,2022-05-15 12:46:10.192,526,40


In [16]:
chart = (
    alt.Chart(
        data,
        title='Raw measurements',
    )
    .mark_point()
    .encode(
        alt.X('reading', scale=alt.Scale(clamp=True), title='Sensor reading'),
        alt.Y('brightness', scale=alt.Scale(clamp=True), title='Brightness setting')
    )
    .interactive()
)
chart.save('raw-measurements.json')
chart

alt.Chart(...)

In [28]:
data['hour'] = [x.hour + x.minute / 60 + x.second / 3600 for x in data['datetime']]
data['hour']

0      9.274167
1     12.726111
2     12.833889
3     14.295278
4     14.708056
        ...    
19    15.209722
20    18.093889
21    12.765556
22    12.769444
23    13.110833
Name: hour, Length: 128, dtype: float64

In [33]:
chart = (
    alt.Chart(
        data,
        title='Raw measurements',
    )
    .mark_point()
    .encode(
        alt.X('reading', scale=alt.Scale(clamp=True), title='Sensor reading'),
        alt.Y('brightness', scale=alt.Scale(clamp=True), title='Brightness setting'),
        alt.Color('hour', scale=alt.Scale(scheme='turbo'), title='Hour'),
        [
            alt.Tooltip('reading', title='Sensor reading'),
            alt.Tooltip('brightness', title='Brightness setting'),
            alt.Tooltip('datetime', title='Date'),
            alt.Tooltip('hour', title='Hour'),
        ]
    )
    .interactive()
)
chart.save('measurements_time.json')
chart

alt.Chart(...)

In [43]:
smoothed = loess.loess_1d.loess_1d(np.array(data['reading']), np.array(data['brightness']))

In [48]:
data["loess_x"], data["loess_y"], data["loess_w"] = smoothed
data["loess_residual"] = data["brightness"] - data["loess_y"]

In [66]:
chart = (
    alt.Chart(
        data,
        title='Loess smoothing',
    )
    .mark_point()
    .encode(
        alt.X('loess_x', scale=alt.Scale(clamp=True), title='Sensor reading'),
        alt.Y('loess_y', scale=alt.Scale(clamp=True), title='Brightness setting'),
        alt.Color('loess_w', scale=alt.Scale(scheme='blues'), title='Weight'),
    )
    .interactive()
)
chart.save('loess.json')
chart

alt.Chart(...)

In [74]:
def model(x, a, b, c):
    return a + b * x + c * x**2

popt, pconv = scipy.optimize.curve_fit(model, data['reading'].to_numpy(), data['brightness'].to_numpy(), p0=[1, 1, 1])

In [75]:
popt

array([ 7.71442008e+00,  2.31382812e-02, -1.65268073e-06])

In [79]:
x = np.linspace(0, 12000, 100)
y = model(x, *popt)
fit_df = pd.DataFrame(dict(x=x, y=y))

In [82]:
chart = (
    alt.Chart(
        fit_df,
        title='Curve fit',
    )
    .mark_line()
    .encode(
        alt.X('x', scale=alt.Scale(clamp=True), title='Sensor reading'),
        alt.Y('y', scale=alt.Scale(clamp=True), title='Brightness setting'),
    )
    .interactive()
)
chart.save('fit.json')
chart

alt.Chart(...)

In [97]:
basex = [0, 200, 400, 500, 700, 900, 1000, 1200, 1500, 3500, 6800, 9000]
basey = [0, 7,   15,  21,  24,  30,  33,   34,   40,   70,   90, 100]
 
model2 = scipy.interpolate.interp1d(basex, basey, kind='quadratic')
x = np.linspace(0, np.max(basex), 100)
y = model2(x)
interpolate_df = pd.DataFrame(dict(x=x, y=y))

chart = (
    alt.Chart(
        interpolate_df,
        title='Interpolation',
    )
    .mark_line()
    .encode(
        alt.X('x', title='Sensor reading'),
        alt.Y('y', title='Brightness setting'),
    )
    .interactive()
)
chart.save('interpolation.json')
chart

alt.Chart(...)